In [66]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import cv2
import mediapipe as mp
import os
import subprocess
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Prepare Data Testing Image 

In [67]:
def list_jpg_png_files(directory):
    jpg_png_files = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(root, filename)
                file_path = file_path.replace('\\', '/')
                jpg_png_files.append(file_path)
    return jpg_png_files

list_jpg_png_files('./datasets/Testing-Image')

['./datasets/Testing-Image/87_kerja_1.jpg',
 './datasets/Testing-Image/8_kerja_1.jpg',
 './datasets/Testing-Image/8_malam_1.jpg',
 './datasets/Testing-Image/8_malam_2.jpg',
 './datasets/Testing-Image/8_siang_2.jpg',
 './datasets/Testing-Image/91_jawab_1.jpg',
 './datasets/Testing-Image/91_kantor_1.jpg',
 './datasets/Testing-Image/92_jawab_1.jpg',
 './datasets/Testing-Image/92_kantor_1.jpg',
 './datasets/Testing-Image/93_jawab_1.jpg',
 './datasets/Testing-Image/93_sore_1.jpg',
 './datasets/Testing-Image/95_bagaimana_1.jpg',
 './datasets/Testing-Image/95_bagaimana_2.jpg',
 './datasets/Testing-Image/96_bagaimana_1.jpg',
 './datasets/Testing-Image/96_besok_1.jpg',
 './datasets/Testing-Image/96_kemarin_2.jpg',
 './datasets/Testing-Image/96_nanti_1.jpg',
 './datasets/Testing-Image/97_lusa_1.jpg',
 './datasets/Testing-Image/97_nanti_1.jpg',
 './datasets/Testing-Image/98_sekarang_2.jpg',
 './datasets/Testing-Image/99_hari_ini1.jpg',
 './datasets/Testing-Image/99_pagi_1.jpg',
 './datasets/Testi

# Test Static Image

In [78]:
def extract_hand_features(image_path):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5
    )

    data = []

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.append(landmark.x)
                landmarks.append(landmark.y)
                landmarks.append(landmark.z)
            data.append([os.path.basename(image_path)] + landmarks)

    hands.close()
    
    return data

sample_data = extract_hand_features('datasets/Testing-Image/ibu.jpg')[0][1:]
sample_data

[0.25624585151672363,
 0.7030981779098511,
 8.205162771446339e-07,
 0.2820224165916443,
 0.67258220911026,
 -0.01174092385917902,
 0.3218759298324585,
 0.6417163014411926,
 -0.03788422793149948,
 0.370854914188385,
 0.6238855123519897,
 -0.06623929738998413,
 0.404569149017334,
 0.6145714521408081,
 -0.09345296770334244,
 0.2556825876235962,
 0.6104653477668762,
 -0.05811608210206032,
 0.30842113494873047,
 0.570963978767395,
 -0.09741616994142532,
 0.343104749917984,
 0.5459408164024353,
 -0.12105543166399002,
 0.3679792284965515,
 0.5252069234848022,
 -0.13478833436965942,
 0.25872889161109924,
 0.6228168606758118,
 -0.07687991112470627,
 0.3196924328804016,
 0.5756872296333313,
 -0.11728732287883759,
 0.3557385802268982,
 0.5481976270675659,
 -0.13338923454284668,
 0.3789525032043457,
 0.5258023142814636,
 -0.14138436317443848,
 0.2728026509284973,
 0.6413312554359436,
 -0.09474864602088928,
 0.3642631769180298,
 0.6205140948295593,
 -0.13181333243846893,
 0.3763650059700012,
 0.635

# Test Predict

In [79]:
import tensorflow as tf

model_path = 'models/12-juni-0130pm.h5'
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                4096      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 66)                2178      
                                                                 
Total params: 8354 (32.63 KB)
Trainable params: 8354 (32.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
import numpy as np 

prepare_input_data = np.expand_dims(sample_data, axis=0)
predictions = model.predict(prepare_input_data)

1/1 [==============================] - 0s 48ms/step


In [81]:
predicted_class_index = np.argmax(predictions)
predicted_class_index

62

In [82]:
class_labels = ['A', 'Aku', 'Apa', 'B', 'Bagaimana', 'Baik', 'Bapak', 'Berapa',
       'Besok', 'C', 'D', 'Dia', 'Dimana', 'E', 'F', 'G', 'H', 'Halo',
       'Hari ini', 'I', 'Ibu', 'J', 'Jawab', 'K', 'Kalian', 'Kamu',
       'Kantor', 'Kapan', 'Kemana', 'Kemarin', 'Kerja', 'L', 'Lelah',
       'Lusa', 'M', 'Maaf', 'Makan', 'Malam', 'Mengapa', 'N', 'Nanti',
       'O', 'P', 'Pagi', 'Q', 'R', 'S', 'Sabar', 'Sakit', 'Sama - sama',
       'Sedih', 'Sekarang', 'Senang', 'Siang', 'Siapa', 'Sore', 'T',
       'Terima kasih', 'Tidur', 'Tolong', 'U', 'V', 'W', 'X', 'Y', 'Z']


predicted_class = class_labels[predicted_class_index]
predicted_class

'W'

# DIEEEEEEEEEEEEEEE

# Function For Testing Directory